In [ ]:
import scanpy as sc
import sciduck as sd

## Load in raw data

In [ ]:
adata = ad.read_h5ad("/path/to/h5ad")

## Standard scanpy preprocessing

In [ ]:
## Normalize the count matrix after storing the raw counts in the raw slot
adata.raw = adata
sc.pp.highly_variable_genes(adata, n_top_genes=4000, subset=False, layer=None, flavor='seurat_v3', batch_key='donor_name')
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

## Calculate some basic QC metrics around UMI and gene counts
sc.pp.calculate_qc_metrics(adata, inplace=True)

## Compute clusters for use later
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=True)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.leiden(adata, flavor="igraph", n_iterations=2)

## Basic quality control (`sciduck`)

In [ ]:
## Filter cells/nuclei on UMI and gene count thresholds, showing the default values.
sd.qc.add_range_constraint(adata, column="counts", gt=2000, lt=100000)
sd.qc.add_range_constraint(adata, column="genes", gt=1000, lt=13000)

## Apply the constraints
sd.qc.apply_constraints(adata)
adata.obs.keeper_cells.value_counts()

Now lets perform quality control using additional quality control metrics such as doublet_score.

In [ ]:
## Filter cells/nuclei on mitochondrial gene expression, showing the default values.
sd.qc.add_range_constraint(adata, column="doublet_score", lt = 0.3)
sd.qc.add_range_constraint(adata, column="pct_counts_mt", lt = 3.0)
sd.qc.add_range_constraint(adata, column="GEX_Reads_mapped_confidently_to_genome", gt = 0.0)
sd.qc.add_range_constraint(adata, column="GEX_Reads_mapped_to_genome", gt = 0.0)
sd.qc.add_range_constraint(adata, column="GEX_Reads_with_TSO", lt = 1.0)

## Apply the constraints
sd.qc.apply_constraints(adata)
adata.obs.keeper_cells.value_counts()

## Quality control involving coarse labels (Neuron / Non-neuron)

In [ ]:
## Neuron / Non-Neuron QC constraints
sd.qc.add_range_constraint(adata, 
                        column = "genes", ## What does the contraint apply to?
                        gt = 2000, ## Only keep cells with more than 2000 genes
                        lt = 13000, ## Only keep cells with less than 13000 genes
                        subset = "Class", 
                        subset_values = ['Excitatory', 'Inhibitory'])
sd.qc.add_range_constraint(adata, 
                        column = "genes", 
                        gt = 1000, ## Only keep cells with more than 1000 genes
                        lt = 13000, ## Only keep cells with less than 1e6 genes
                        subset = "Class", 
                        subset_values = ['Astrocytes', 'Oligodendrocytes', 'Microglia', 'Endothelial', 'Pericytes'])

## Apply the constraints
sd.qc.apply_constraints(adata)
adata.obs.keeper_cells.value_counts()

## Quality control involving clusters

In [ ]:
## Lets use our clusters to perform quality control based on cluster median values
## It's helpful to have visualized the cluster median valudes before setting these thresholds
sd.qc.add_group_level_constraint(adata, 
                                    column="doublet_score", 
                                    groupby = "leiden", 
                                    lt=0.2, 
                                    agg_func = "median")
sd.qc.add_group_level_constraint(adata, 
                                    column="total_counts", 
                                    groupby = "leiden", 
                                    lt=2e5, 
                                    agg_func = "median")

## Apply the constraints
sd.qc.apply_constraints(adata)
adata.obs.keeper_cells.value_counts()